# OmniIndex
## How to use the omniindex connector for python
examples of each api call are provided in the following notebook cells

written and maintained by: [**@james stanbridge**](mailto:james@omniindex.io)

In [1]:
# mandatory cell
# expects omniindex==0.1.12 minimum
import json
from omniindex.api import OmniIndexClient # this will change in future versions to omniindex.connector
import os
import pandas as pd
import time


if you want to use another method to set your environment variables, you can do so. This is the recommomended method for use with jupyter notebooks

In [2]:
SERVER = os.environ['OI_API_TEST_SERVER'] # maps to server
API_KEY = os.environ['OI_API_TEST_API_KEY'] # maps to api_key
UNIT_NAME = os.environ['OI_API_TEST_UNIT_NAME']# maps to unit_name
USER = os.environ['OI_API_TEST_USER'] # maps to user
#
BLOCK_TYPE = "Owner"
MASTER_KEY = os.environ['OI_API_MASTER_KEY_DEMO'] # master key for demonstration

APIserver = "https://api.omniindex.xyz/api_v1"

Setting up the client:
as with all the functions, you need to start with a client object = OmniIndexClient()

Parameters
- server (str) – The server to connect to. This is the node server for your blockchain
- api_key (str) – The api key also know as password for your blockchain
- unit_name (str) – The unit name for your blockchain
- user (str) – The user name for your blockchain (paired with the api key)
- block_type (str) – The block type you want to get the schematic for (typically 'Owner' or 'Global')

## get_block_schematic
it's useful to know the structure of the block you're working with. This function will return the block schematic for a given block id

```python
get_block_schematic()
```

In [3]:
client = OmniIndexClient(SERVER,API_KEY,UNIT_NAME, BLOCK_TYPE, USER, APIserver)
getblock = client.get_block_schematic()
getblock


'{\n"0" : {\n\n"column_name": "addressowners",\n"data_type": "text"\n},\n"1" : {\n\n"column_name": "addresssearchableowners",\n"data_type": "text"\n},\n"2" : {\n\n"column_name": "cc_exp",\n"data_type": "text"\n},\n"3" : {\n\n"column_name": "contentsearchableowners",\n"data_type": "text"\n},\n"4" : {\n\n"column_name": "context",\n"data_type": "text"\n},\n"5" : {\n\n"column_name": "context2",\n"data_type": "text"\n},\n"6" : {\n\n"column_name": "creditowners",\n"data_type": "text"\n},\n"7" : {\n\n"column_name": "creditsearchableowners",\n"data_type": "text"\n},\n"8" : {\n\n"column_name": "dateadded",\n"data_type": "text"\n},\n"9" : {\n\n"column_name": "dobowners",\n"data_type": "text"\n},\n"10" : {\n\n"column_name": "dobsearchableowners",\n"data_type": "text"\n},\n"11" : {\n\n"column_name": "emoji",\n"data_type": "text"\n},\n"12" : {\n\n"column_name": "hash",\n"data_type": "character varying"\n},\n"13" : {\n\n"column_name": "infoowners",\n"data_type": "text"\n},\n"14" : {\n\n"column_name"

## run_analytic_query
```python
run_analytic_query(show_protected, query)
```

1. Cell shows what a SQL select * query looks like - note that the show_protected parameter is set to true, which means you will get the protected fields unredacted
2. Cell shows how to SQL select a COUNT of the number of records in the block

In [6]:
client = OmniIndexClient(SERVER,API_KEY,UNIT_NAME, BLOCK_TYPE, USER, APIserver)
query = client.run_analytic_query("true", "select * from limit 10 ")
query

'{ "results" : [\n{\n"address" : "8629 Yaman Cliffs Apt. 933\\nZenginland, MO 37645","cc_exp" : "02/29","context" : "","context2" : "","credit" : "4856724439645834546","dateadded" : "2023-04-28 10:50:17","dob" : "1967-09-04","emoji" : "\\ud83d\\udc68\\ud83c\\udffe\\u200d\\ud83e\\udd1d\\u200d\\ud83d\\udc68\\ud83c\\udfff","hash" : "D50F0C42E0FED35D7CCC6CE416F18C09F1414FF676AE72AD3A665A0F3D6F8365","info" : "Amet provident hic dignissimos et. Quas et possimus reiciendis tempore. Molestiae atque deleniti. Odio mollitia hic at optio. Ipsam reprehenderit sequi tenetur facere nostrum labore. Ipsam et unde expedita minima nihil. Qui cumque facere voluptatibus nam consequuntur corporis. Sed animi ex laborum harum quia quas. Repudiandae repellendus eaque. Impedit labore facilis.","ipv4" : "74.135.168.242","job" : "Adli tabip","name" : "Hudavent \\u00c7akar Man\\u00e7o Demir","oidxid" : "1","phone" : "+44(0)808 157 0656","prevhash" : "","priorhash" : "eve","sentiment" : "","sentiment2" : "","uuid"

In [5]:
client = OmniIndexClient(SERVER,API_KEY,UNIT_NAME, BLOCK_TYPE, USER, APIserver)
query = client.run_analytic_query("true", "select count (*) from ")
print(query)

{ "results" : [
{
"count" : "14"}
]
}


generating fake-random data for the examples


In [7]:
from faker import Faker
from faker.providers import internet
""" Creates a fake info for testing purposes
    :param lcid: language codes for localisation
    :type lcid: list
    :return: 
    """

def create_info(lcid: list):
    # Create a Faker object
    fake = Faker(lcid)
    
    # Generate random data
    NAME = fake.name()
    ADDRESS = fake.address()
    INFO = fake.paragraph(nb_sentences=10, variable_nb_sentences=False)
    IPV4 = fake.ipv4()
    CREDIT = fake.credit_card_number()
    CC_EXP = fake.credit_card_expire()
    DOB = str(fake.date_of_birth())
    UUID = fake.uuid4()
    EMOJI = fake.emoji()
    JOB = fake.job()
    PHONE = fake.phone_number()
    USER_AGENT = fake.user_agent()
    
    # Create a JSON object
    data = {
        "nameEncrypt": NAME,
        "addressEncrypt": ADDRESS,
        "infoEncrypt": INFO,
        "ipv4Encrypt": IPV4,
        "creditEncrypt": CREDIT,
        "cc_exp": CC_EXP,
        "dobEncrypt": DOB,
        "uuid": UUID,
        "emoji": EMOJI,
        "job": JOB,
        "phone": PHONE,
        "user_agent": USER_AGENT
    }
    
    return data


    

In [8]:
data = create_info(['tr_TR', 'en_GB', 'ja_JP'])
data

{'nameEncrypt': 'Douglas Pearson',
 'addressEncrypt': '2546 Yaman Glens Suite 103\nTezolside, GA 53888',
 'infoEncrypt': 'Aperiam alias blanditiis debitis magni eaque delectus. Reiciendis magni unde quidem deserunt. Repudiandae quod quisquam deserunt vero accusamus. Qui facilis mollitia numquam. Excepturi error architecto dolorum. Odit excepturi dolorum. Repellat repellendus ad nobis a dolore ratione. Autem voluptatum tempore quidem dolores dolor. Suscipit animi totam. Saepe dolores nobis labore culpa autem aperiam.',
 'ipv4Encrypt': '49.133.240.54',
 'creditEncrypt': '2241539371018310',
 'cc_exp': '12/29',
 'dobEncrypt': '1960-06-21',
 'uuid': '7cab36d2-d95d-416c-b5d1-9480a8cc4ac2',
 'emoji': '🦅',
 'job': 'Engineer, energy',
 'phone': '090-3466-5572',
 'user_agent': 'Opera/8.53.(X11; Linux i686; yue-HK) Presto/2.9.162 Version/12.00'}

## minedata

Adds a new block to the blockchain.
- first cell creates a new, random data payload
- second cell adds the data to the blockchain

In [9]:
# This cell creates a payload (JSON file of records to be added)

# it is recommended practice to always add a timestamp to your data
DATE_ADDED = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
import datetime

# OmniIndex JSON parser accepts only string values
STAMP = str(datetime.datetime.now().timestamp() * 1000) 

# setup a payload to add to the block
query = create_info(['tr_TR', 'en_GB', 'ja_JP']) # added some localised data just for fun
query["dateAdded"] = DATE_ADDED
payload = json.dumps(query)
print(f"payload = {payload}")

payload = {"nameEncrypt": "\u00c7ev. \u00c7\u0131dal Ziyaettin \u00d6calan", "addressEncrypt": "777 Kocaba\u015f Freeway Apt. 395\nAtanurmouth, ME 41070", "infoEncrypt": "Neque provident quia ut eaque nemo. Doloremque repudiandae perferendis cumque consequatur iste. Distinctio voluptas unde fuga minus suscipit illum voluptate. Nisi optio sint autem provident ipsum officia. Quasi quo quam aliquid. Atque est blanditiis iusto quidem nisi ad. Nobis aut non delectus aspernatur totam ad. Minus quae eaque eligendi odio doloribus. Cumque omnis esse repudiandae nam vero. Autem cumque a id iure.", "ipv4Encrypt": "37.102.101.215", "creditEncrypt": "6531684897804148", "cc_exp": "04/24", "dobEncrypt": "1971-03-23", "uuid": "97923fdc-4969-4270-a85e-4bf87989e6bc", "emoji": "\ud83e\uddd1\u200d\ud83c\udf73", "job": "Ambalajc\u0131", "phone": "090-4868-7534", "user_agent": "Opera/9.63.(Windows 98; sat-IN) Presto/2.9.174 Version/10.00", "dateAdded": "2023-05-12 16:30:55"}


In [10]:

# create a client object
client = OmniIndexClient(SERVER, API_KEY, UNIT_NAME, BLOCK_TYPE, USER, APIserver)
result = client.post_minedata(MASTER_KEY, payload,)
print(f"This is the return value: {result}")

This is the return value: 


# APPENDIX
## useful tools etc

Recipes for generating usernames and passwords

In [ ]:
# Generate a ten-character alphanumeric password with at least one lowercase character, at least one uppercase character, and at least three digits:
from faker import Faker


def ten_pass():
    fake = Faker()
    password = fake.password(length=10, special_chars=True, digits=True, upper_case=True, lower_case=True)
    return password

def user_name():
    fake = Faker()
    user_name = fake.user_name()
    return user_name

def pass_key():
    fake = Faker()
    api_key = fake.password(length=32, special_chars=True, digits=True, upper_case=True, lower_case=True)
    return api_key

new_user = user_name()
new_pass = ten_pass()
new_key = pass_key()

print(f"new username: {new_user}")
print(f"new password: {new_pass}")
print(f"new api key: {new_key}")
